In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install neurokit2
!pip install biosppy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.2/689.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.1 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import biosppy.signals.ecg as ecg
import numpy as np
import neurokit2 as nk
from scipy.signal import butter, filtfilt
from scipy import signal

path = "/content/drive/MyDrive/DREAMER.mat"
raw = sio.loadmat(path)


In [ ]:
import numpy as np
import pandas as pd
from scipy import signal, stats
from sklearn.preprocessing import StandardScaler

def preprocess_EEG_time_domain(raw, feature):
    # Calculate time-domain features
    mean = np.mean(raw)
    std_dev = np.std(raw)
    skewness = stats.skew(raw)
    kurtosis = stats.kurtosis(raw)

    feature.append(mean)
    feature.append(std_dev)
    feature.append(skewness)
    feature.append(kurtosis)
    return feature

def feat_extract_EEG_time_domain(raw):
    num_participants = 23
    num_videos = 18
    num_channels = 14
    num_features = 4  # Mean, Std Dev, Skewness, Kurtosis
    EEG_tmp = np.zeros((num_participants, num_videos, num_channels * num_features))

    for participant in range(num_participants):
        for video in range(num_videos):
            for i in range(num_channels):
                B, S = [], []
                basl = (raw["DREAMER"][0, 0]["Data"]
                        [0, participant]["EEG"][0, 0]
                        ["baseline"][0, 0][video, 0][:, i])
                stim = (raw["DREAMER"][0, 0]["Data"]
                        [0, participant]["EEG"][0, 0]
                        ["stimuli"][0, 0][video, 0][:, i])
                B = preprocess_EEG_time_domain(basl, B)
                S = preprocess_EEG_time_domain(stim, S)
                Extrod = np.divide(S, B)
                EEG_tmp[participant, video, num_features*i:num_features*(i+1)] = Extrod

    col = []
    for i in range(num_channels):
        col.append(f"mean_{i+1}_un")
        col.append(f"std_dev_{i+1}_un")
        col.append(f"skewness_{i+1}_un")
        col.append(f"kurtosis_{i+1}_un")

    data_EEG = pd.DataFrame(EEG_tmp.reshape((num_participants * num_videos,
                                              EEG_tmp.shape[2])), columns=col)
    scaler = StandardScaler()
    for i in range(len(col)):
        data_EEG[col[i][:-3]] = scaler.fit_transform(data_EEG[[col[i]]])
    data_EEG.drop(col, axis=1, inplace=True)
    return data_EEG

def participant_affective(raw):
    a = np.zeros((23, 18, 7), dtype=object)
    for participant in range(0, 23):
        for video in range(0, 18):
            a[participant, video, 0] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Age"][0][0][0])

            a[participant, video, 1] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Gender"][0][0][0])

            a[participant, video, 2] = int(participant+1)

            a[participant, video, 3] = int(video+1)

            a[participant, video, 4] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreValence"]
                                           [0, 0][video, 0])
            a[participant, video, 5] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreArousal"]
                                           [0, 0][video, 0])
            a[participant, video, 6] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreDominance"]
                                           [0, 0][video, 0])
    b = pd.DataFrame(a.reshape((23*18, a.shape[2])),
                     columns=["age", "gender", "participant",
                              "video", "valence", "arousal", "dominance"])
    return b

# Assume 'raw' is already loaded with the necessary EEG data
df_features_time_domain = feat_extract_EEG_time_domain(raw)
df_participant_affective = participant_affective(raw)

df_participant_affective["valence"] = df_participant_affective["valence"].astype(int)
df_participant_affective["arousal"] = df_participant_affective["arousal"].astype(int)
df_participant_affective["dominance"] = df_participant_affective["dominance"].astype(int)
df = pd.concat([df_features_time_domain, df_participant_affective], axis=1)

data = df.copy()
data



,mean_1,std_dev_1,skewness_1,kurtosis_1,mean_2,std_dev_2,skewness_2,kurtosis_2,mean_3,std_dev_3,...,std_dev_14,skewness_14,kurtosis_14,age,gender,participant,video,valence,arousal,dominance
0,0.078972,-0.278388,-0.020604,-0.069875,0.185863,-0.322584,-0.643325,0.047408,0.546392,-0.472850,...,-0.457287,0.211842,-0.116104,22,male,1,1,4,3,2
1,-0.070469,-0.110870,-0.007740,-0.077030,0.071368,0.023180,0.232760,-0.033932,-0.044767,0.769401,...,-0.620344,0.005617,-0.022834,22,male,1,2,3,3,1
2,-0.228583,-0.085445,-0.036057,-0.069336,-0.119083,0.021121,-1.098617,0.047176,0.170309,0.163278,...,0.011738,-5.740220,-0.105603,22,male,1,3,5,4,4
3,-0.318846,-0.201742,-0.015337,-0.066728,-0.342092,-0.126448,-0.350559,0.047277,0.234335,-0.231640,...,-0.405401,-0.009565,-0.137296,22,male,1,4,4,3,2
4,-0.298467,-0.086567,-0.026699,-0.064326,-0.195509,-0.133407,0.338947,0.050546,4.002156,6.143943,...,0.015716,0.034256,-0.123908,22,male,1,5,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.785470,-0.348197,-0.015725,-0.058063,1.121600,-0.321252,-0.176606,0.048335,0.311117,-0.463029,...,-0.108512,-0.059899,-0.064152,25,male,23,14,2,2,2
410,0.022241,-0.053556,-0.038978,-0.049065,0.011893,0.033432,-1.165585,1.523263,-0.300072,-0.143479,...,0.462141,0.065810,0.081583,25,male,23,15,2,2,2
411,-0.253299,-0.166956,-0.030465,-0.068856,0.755750,0.039908,-0.100242,0.046887,2.285044,-0.269243,...,0.639588,0.561941,0.552946,25,male,23,16,3,3,2
412,-0.236578,-0.406442,-0.038409,-0.069762,0.715725,0.153486,-0.345873,0.046997,-0.252190,-0.243378,...,-0.154659,0.829981,-0.095888,25,male,23,17,2,2,4


In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with the necessary columns

# Define a function to convert values to binary
def to_binary(value):
    return 1 if value > 1 else 0

print("Unique values before transformation:")
print("Valence:", df["valence"].unique())
print("Arousal:", df["arousal"].unique())
print("Dominance:", df["dominance"].unique())

# Apply this function to the "valence", "arousal", and "dominance" columns
data["valence"] = df["valence"].apply(to_binary)
data["arousal"] = df["arousal"].apply(to_binary)
data["dominance"] = df["dominance"].apply(to_binary)

# Verify the changes
print(data[["valence", "arousal", "dominance"]].head())

data

Unique values before transformation:
Valence: [4 3 5 1 2]
Arousal: [3 4 2 1 5]
Dominance: [2 1 4 5 3]
   valence  arousal  dominance
0        1        1          1
1        1        1          0
2        1        1          1
3        1        1          1
4        1        1          1


,mean_1,std_dev_1,skewness_1,kurtosis_1,mean_2,std_dev_2,skewness_2,kurtosis_2,mean_3,std_dev_3,...,std_dev_14,skewness_14,kurtosis_14,age,gender,participant,video,valence,arousal,dominance
0,0.078972,-0.278388,-0.020604,-0.069875,0.185863,-0.322584,-0.643325,0.047408,0.546392,-0.472850,...,-0.457287,0.211842,-0.116104,22,male,1,1,1,1,1
1,-0.070469,-0.110870,-0.007740,-0.077030,0.071368,0.023180,0.232760,-0.033932,-0.044767,0.769401,...,-0.620344,0.005617,-0.022834,22,male,1,2,1,1,0
2,-0.228583,-0.085445,-0.036057,-0.069336,-0.119083,0.021121,-1.098617,0.047176,0.170309,0.163278,...,0.011738,-5.740220,-0.105603,22,male,1,3,1,1,1
3,-0.318846,-0.201742,-0.015337,-0.066728,-0.342092,-0.126448,-0.350559,0.047277,0.234335,-0.231640,...,-0.405401,-0.009565,-0.137296,22,male,1,4,1,1,1
4,-0.298467,-0.086567,-0.026699,-0.064326,-0.195509,-0.133407,0.338947,0.050546,4.002156,6.143943,...,0.015716,0.034256,-0.123908,22,male,1,5,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.785470,-0.348197,-0.015725,-0.058063,1.121600,-0.321252,-0.176606,0.048335,0.311117,-0.463029,...,-0.108512,-0.059899,-0.064152,25,male,23,14,1,1,1
410,0.022241,-0.053556,-0.038978,-0.049065,0.011893,0.033432,-1.165585,1.523263,-0.300072,-0.143479,...,0.462141,0.065810,0.081583,25,male,23,15,1,1,1
411,-0.253299,-0.166956,-0.030465,-0.068856,0.755750,0.039908,-0.100242,0.046887,2.285044,-0.269243,...,0.639588,0.561941,0.552946,25,male,23,16,1,1,1
412,-0.236578,-0.406442,-0.038409,-0.069762,0.715725,0.153486,-0.345873,0.046997,-0.252190,-0.243378,...,-0.154659,0.829981,-0.095888,25,male,23,17,1,1,1


In [ ]:
valence_zero_count = (data["valence"] == 0).sum()
arousal_zero_count = (data["arousal"] == 0).sum()
dominance_zero_count = (data["dominance"] == 0).sum()
print(f"Number of 0 values in valence: {valence_zero_count}")
print(f"Number of 0 values in arousal: {arousal_zero_count}")
print(f"Number of 0 values in dominance: {dominance_zero_count}")

valence_one_count = (data["valence"] == 1).sum()
arousal_one_count = (data["arousal"] == 1).sum()
dominance_one_count = (data["dominance"] == 1).sum()
print(f"Number of 0 values in valence: {valence_one_count}")
print(f"Number of 0 values in arousal: {arousal_one_count}")
print(f"Number of 0 values in dominance: {dominance_one_count}")

Number of 0 values in valence: 77
Number of 0 values in arousal: 21
Number of 0 values in dominance: 14
Number of 0 values in valence: 337
Number of 0 values in arousal: 393
Number of 0 values in dominance: 400


In [ ]:
data.isnull().sum().sum()

0

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select columns for normalization (excluding non-numeric columns like 'participant', 'video', etc.)
columns_to_normalize = data.columns.difference(['participant', 'video', 'gender', 'valence', 'arousal', 'dominance'])

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the data and transform the selected columns
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
data['gender'].replace({'male': 1, 'female': 0}, inplace=True)
data

,mean_1,std_dev_1,skewness_1,kurtosis_1,mean_2,std_dev_2,skewness_2,kurtosis_2,mean_3,std_dev_3,...,std_dev_14,skewness_14,kurtosis_14,age,gender,participant,video,valence,arousal,dominance
0,0.352661,0.063970,0.175631,0.551405,0.349813,0.031666,0.471462,0.911529,0.334544,0.072836,...,0.049777,0.668790,0.327112,0.000000,1,1,1,1,1,1
1,0.338169,0.079764,0.176170,0.551052,0.339828,0.059303,0.536605,0.907832,0.295515,0.201172,...,0.033597,0.659700,0.331639,0.000000,1,1,2,1,1,0
2,0.322836,0.082162,0.174983,0.551432,0.323218,0.059139,0.437608,0.911518,0.309714,0.138554,...,0.096317,0.406438,0.327622,0.000000,1,1,3,1,1,1
3,0.314083,0.071196,0.175852,0.551561,0.303768,0.047343,0.493231,0.911523,0.313942,0.097755,...,0.054926,0.659031,0.326084,0.000000,1,1,4,1,1,1
4,0.316059,0.082056,0.175375,0.551680,0.316552,0.046787,0.544501,0.911671,0.562700,0.756412,...,0.096712,0.660962,0.326733,0.000000,1,1,5,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.421173,0.057388,0.175835,0.551989,0.431422,0.031772,0.506166,0.911571,0.319011,0.073851,...,0.084385,0.656812,0.329634,0.272727,1,23,14,1,1,1
410,0.347160,0.085168,0.174861,0.552433,0.334641,0.060123,0.432628,0.978606,0.278659,0.106863,...,0.141009,0.662353,0.336707,0.272727,1,23,15,1,1,1
411,0.320439,0.074476,0.175218,0.551456,0.399515,0.060640,0.511844,0.911505,0.449333,0.093870,...,0.158617,0.684221,0.359585,0.272727,1,23,16,1,1,1
412,0.322061,0.051896,0.174885,0.551411,0.396024,0.069719,0.493580,0.911510,0.281820,0.096543,...,0.079806,0.696036,0.328093,0.272727,1,23,17,1,1,1


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.feature_selection import mutual_info_classif
from imblearn.over_sampling import RandomOverSampler

In [ ]:
def Feature_Selection(data, threshold):
    y = data.valence
    X = data.drop('valence', axis=1)
    importance = mutual_info_classif(X, y)
    feat_importance = pd.Series(importance, index=X.columns)
    feat_importance = feat_importance.sort_values(ascending=False)
    selected_features = [i for i in feat_importance.index if feat_importance[i] > threshold]
    df_selected_features = data[selected_features + ['valence']]
    return df_selected_features

# Apply feature selection
EEG_selected_features = Feature_Selection(data, 0.001)

# Fix the copy issue by using .loc
EEG_selected_features.loc[:, "valence"] = data.valence

# Handle resampling
X = EEG_selected_features.drop('valence', axis=1)
y = EEG_selected_features['valence']
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)


from collections import Counter

# Before oversampling
print("Class Distribution Before Oversampling:")
print(Counter(y))

# After oversampling
print("\nClass Distribution After Oversampling:")
print(Counter(y_resampled))


Class Distribution Before Oversampling:
Counter({1: 337, 0: 77})

Class Distribution After Oversampling:
Counter({1: 337, 0: 337})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Convert to numpy arrays and ensure the correct data type
X_train = X_train.to_numpy().astype('float32')
X_test = X_test.to_numpy().astype('float32')
y_train = y_train.to_numpy().astype('int32')
y_test = y_test.to_numpy().astype('int32')

# Add a dimension for CNN and RNN input
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for RNN
X_train_rnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_rnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [ ]:
!pip install pandas numpy tensorflow scikit-learn imblearn openpyxl

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import mutual_info_classif
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
import time
from keras.optimizers import Adam

In [ ]:
# Define deep learning models
def create_cnn(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(input_shape, 1)))
    model.add(Conv1D(32, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_dnn(input_shape):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_rnn(input_shape):
    model = Sequential()
    model.add(LSTM(128, input_shape=(input_shape, 1), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, epochs=20, batch_size=32):
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
    training_time = time.time() - start_time

    y_pred = (model.predict(X_test) > 0.5).astype("int32")

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    return {
        'history': history,
        'training_time': training_time,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Create and train models
cnn_model = create_cnn(X_train_cnn.shape[1])
dnn_model = create_dnn(X_train.shape[1])
# Create and train RNN model
rnn_model = create_rnn(X_train_rnn.shape[1])



In [ ]:
print("Training CNN...")
cnn_results = train_and_evaluate_model(cnn_model, X_train_cnn, y_train, X_test_cnn, y_test)

print("Training DNN...")
dnn_results = train_and_evaluate_model(dnn_model, X_train, y_train, X_test, y_test)

print("Training RNN...")
rnn_results = train_and_evaluate_model(rnn_model, X_train_rnn, y_train, X_test_rnn, y_test)

# Save the results to an Excel file
results = [
    ["CNN", cnn_results['accuracy'] * 100, cnn_results['precision'] * 100, cnn_results['recall'] * 100, cnn_results['f1'] * 100, cnn_results['training_time']],
    ["DNN", dnn_results['accuracy'] * 100, dnn_results['precision'] * 100, dnn_results['recall'] * 100, dnn_results['f1'] * 100, dnn_results['training_time']],
    ["RNN", rnn_results['accuracy'] * 100, rnn_results['precision'] * 100, rnn_results['recall'] * 100, rnn_results['f1'] * 100, rnn_results['training_time']]
]

df_results = pd.DataFrame(results, columns=['Model', 'Test Accuracy', 'Precision', 'Recall', 'F1 Score', 'Training Time (seconds)'])
output_file = 'Dl_time_valence_binary_results.xlsx'
df_results.to_excel(output_file, index=False)

print("Evaluation results saved to:", output_file)

Training CNN...
Epoch 1/20
15/15 [==============================] - 6s 80ms/step - loss: 0.6909 - accuracy: 0.5541 - val_loss: 0.6993 - val_accuracy: 0.4532
Epoch 2/20
15/15 [==============================] - 0s 24ms/step - loss: 0.6845 - accuracy: 0.5499 - val_loss: 0.6988 - val_accuracy: 0.5320
Epoch 3/20
15/15 [==============================] - 0s 18ms/step - loss: 0.6780 - accuracy: 0.6454 - val_loss: 0.6920 - val_accuracy: 0.5468
Epoch 4/20
15/15 [==============================] - 0s 10ms/step - loss: 0.6665 - accuracy: 0.6221 - val_loss: 0.6746 - val_accuracy: 0.5567
Epoch 5/20
15/15 [==============================] - 0s 10ms/step - loss: 0.6499 - accuracy: 0.6454 - val_loss: 0.6691 - val_accuracy: 0.5665
Epoch 6/20
15/15 [==============================] - 0s 10ms/step - loss: 0.6438 - accuracy: 0.6306 - val_loss: 0.6827 - val_accuracy: 0.5419
Epoch 7/20
15/15 [==============================] - 0s 10ms/step - loss: 0.6366 - accuracy: 0.6497 - val_loss: 0.6431 - val_accuracy: 0.58